In [1]:
import os
import gzip
import pandas as pd
import sys
from io import StringIO

In [2]:
def add_project_root_to_sys_path(target_file="config.py"):
    current_path = os.getcwd()
    while True:
        if target_file in os.listdir(current_path):
            # 找到包含 config.py 的目录，加入 sys.path
            if current_path not in sys.path:
                sys.path.append(current_path)
            break
        else:
            # 向上一级目录查找
            parent_path = os.path.dirname(current_path)
            if parent_path == current_path:
                # 到达根目录还没找到，停止
                raise FileNotFoundError(f"未找到包含 {target_file} 的目录")
            current_path = parent_path

add_project_root_to_sys_path()

In [10]:
from config import breast_raw_data_path, breast_expression_probe, breast_expression_gene

In [4]:
# 读取数据
raw_data = breast_raw_data_path
gpl_data = os.path.join(raw_data, "GPL13607_old_annotations.txt.gz")

In [5]:
# GSM
gsm_files = [
    fn for fn in os.listdir(raw_data)
    if fn.startswith("GSM") and fn.endswith(".txt.gz")
]

In [6]:
exam_path = raw_data + "/GSM1823774_252800416863_S01_GE1_107_Sep09_1_1.txt.gz"

with gzip.open(exam_path, "rt") as fp:
    for i, line in enumerate(fp):
        if i >= 10:  # 只看前 10 行
            break
        print(f"{i:02d}: {line.strip()}")

00: TYPE	text	text	text	text	integer	float	float	text	text	text	integer	integer	integer	integer	float	float	float	float	float	float	text	text	text	text	text	text	text	text	text	text	text	text	integer	integer	text	integer	integer	integer	float	float	float	float	float	float	float	float	integer	integer	float	integer	float	float	float	float	integer	float	integer	float	integer	text	integer	integer	float	float	integer	float	float	float	float	float	float	float	integer	integer	integer	float	float	integer	float	float	integer	float	integer	integer	float	integer	integer	text	integer	float	float	integer	float	float	float	float	integer	integer	float	float	integer	integer	integer	integer	integer	float	integer	integer	float	integer	integer	integer	integer	float	float	integer	integer	integer	integer	integer	integer	integer	float	float	integer	float	integer	integer	float	integer	float	integer	float	float	float	integer	text	integer	integer	float	float	float	float	float	float	integer	integer	float	text	i

In [7]:
# header 在第 9 行（0-based）
skiprows = 9  

expr_list = []
for fn in gsm_files:
    sample = fn.split("_")[0]
    path = os.path.join(raw_data, fn)
    df = pd.read_csv(
        path,
        sep="\t",
        compression="gzip",
        skiprows=skiprows,       # 跳过前 9 行
        header=0,                # 第 10 行作为列名
        usecols=["ProbeName", "gProcessedSignal"],
        dtype={"ProbeName": str, "gProcessedSignal": float}
    ).rename(columns={
        "ProbeName": "ProbeID",
        "gProcessedSignal": sample
    }).set_index("ProbeID")
    expr_list.append(df)

In [8]:
# 横向拼接成 探针 × 样本 矩阵
expr_probe = pd.concat(expr_list, axis=1)
expr_probe.index.name = "ProbeID"

In [9]:
print("探针矩阵大小：", expr_probe.shape)
expr_probe.head()

探针矩阵大小： (62976, 296)


,GSM1823702,GSM1823703,GSM1823704,GSM1823705,GSM1823706,GSM1823707,GSM1823708,GSM1823709,GSM1823710,GSM1823711,...,GSM1823988,GSM1823989,GSM1823990,GSM1823991,GSM1823992,GSM1823993,GSM1823994,GSM1823995,GSM1823996,GSM1823997
ProbeID,,,,,,,,,,,,,,,,,,,,,
GE_BrightCorner,43873.100000,103931.30000,107175.70000,40539.770000,51851.260000,27969.170000,11505.380000,28142.490000,47091.850000,103723.100000,...,48539.670000,121718.00000,62897.060000,58973.430000,49363.660000,56851.020000,209177.600000,236966.100000,70396.700000,71913.090000
DarkCorner,4.379649,12.30392,17.38047,3.428378,3.922062,3.058394,2.287393,2.700101,7.956330,7.321879,...,4.910917,10.72623,9.329732,3.668819,3.444742,4.168148,7.319132,3.399353,3.676441,3.890050
DarkCorner,4.442070,12.45070,25.43444,3.448140,3.950736,3.075234,2.301809,2.714079,8.054805,7.409429,...,3.280390,10.80686,9.389660,3.709740,3.478230,4.196881,3.776748,3.430884,8.151698,3.919043
A_23_P326296,375.943500,1045.56500,874.59330,529.552900,603.959600,533.085400,194.138100,575.882700,425.315700,155.154300,...,749.563100,941.56760,1048.177000,1037.725000,391.805100,811.717500,551.474900,720.586000,322.359000,800.101800
A_24_P287941,176.308600,177.71100,392.02470,202.562600,234.638700,231.480100,53.631490,206.489200,84.932630,78.384590,...,207.226000,439.01850,314.233800,327.821500,189.999000,508.149200,527.609800,271.967900,314.362100,301.176700


In [11]:
gpl_file = raw_data + "/GPL13607_old_annotations.txt.gz"

# 找到 header 行（第一行字段数 > 3）
with gzip.open(gpl_file, "rt") as f:
    all_lines = list(f)

# 读取数据（从 header 开始）
for i, line in enumerate(all_lines):
    if not line.startswith("!") and line.count("\t") >= 3:
        header_index = i
        break
        
data_lines = all_lines[header_index:]

gpl = pd.read_csv(
    StringIO("".join(data_lines)),
    sep="\t",
    dtype=str,
    on_bad_lines='skip'
)

print("✔ 读取成功，字段有：", gpl.columns.tolist())

✔ 读取成功，字段有： ['ID', 'ProbeName', 'GB_ACC', 'ControlType', 'accessions', 'GeneName', 'Description', 'chr_coord', 'SEQUENCE', 'SPOT_ID']


In [12]:
# 提取 ID 和基因名，并重命名
gpl = gpl[["ID", "GeneName"]].rename(columns={
    "ID": "ProbeID",
    "GeneName": "Gene"
})

# 删除无效（未注释）行
gpl = gpl.dropna(subset=["Gene"])

# 显示有效条数
print("✔️ 有效注释条目数：", len(gpl))

✔️ 有效注释条目数： 62976


In [13]:
# 检查哪一列与 expr_probe.index 匹配
probe_ids = set(expr_probe.index)
for col in gpl.columns:
    match_count = gpl[col].isin(probe_ids).sum()
    if match_count > 0:
        pct = match_count / len(expr_probe.index) * 100
        print(f"列 {col} 匹配到 {match_count} 个探针 ({pct:.2f}%)")

列 Gene 匹配到 5916 个探针 (9.39%)


In [14]:
# 读取 GPL 注释文件并打印所有列名，帮助确认应使用哪两列
gpl_path = os.path.join(raw_data, "GPL13607_old_annotations.txt.gz")

with gzip.open(gpl_path, "rt") as fp:
    lines = list(fp)

# 找到表头（即字段名）那一行的下标，通常是以 "ID\t" 开头的那行
for i, l in enumerate(lines):
    if l.startswith("ID\t"):  # 精确定位表头
        header_idx = i
        break

# 表头及表体内容
table_lines = lines[header_idx:]

# 组装成字符串后用 pandas 读取
gpl = pd.read_csv(StringIO("".join(table_lines)), sep="\t", dtype=str)

gpl.head()

,ID,ProbeName,GB_ACC,ControlType,accessions,GeneName,Description,chr_coord,SEQUENCE,SPOT_ID
0,1,GE_BrightCorner,NaN,1,NaN,GE_BrightCorner,NaN,NaN,NaN,--GE_BrightCorner
1,2,DarkCorner,NaN,1,NaN,DarkCorner,NaN,NaN,NaN,--DarkCorner
2,3,DarkCorner,NaN,1,NaN,DarkCorner,NaN,NaN,NaN,--DarkCorner
3,4,A_23_P326296,NM_144987,0,ref|NM_144987|ref|NM_001040425|ens|ENST0000029...,U2AF1L4,ref|Homo sapiens U2 small nuclear RNA auxiliar...,hs|chr19:036235296-036235237,GTATGGGGAGATTGAAGAGATGAATGTGTGCGACAACCTTGGGGAC...,NaN
4,5,A_24_P287941,NM_013290,0,ref|NM_013290|ref|NM_016556|ens|ENST0000039379...,PSMC3IP,ref|Homo sapiens PSMC3 interacting protein (PS...,hs|chr17:040724775-040724716,AAATTGCAGTAGCTTGAGGTTAACATTTAGACTTGGAACAATGCTA...,NaN


In [15]:
# 根据上述匹配结果选择
id_col = "ProbeName"
gene_col = "GeneName"

# 提取并映射，聚合到基因水平
gpl_sub = gpl[[id_col, gene_col]].dropna(subset=[gene_col])
gpl_sub = gpl_sub.rename(columns={id_col: "ProbeID", gene_col: "Gene"})

expr_gene = (
    expr_probe
    .reset_index()
    .merge(gpl_sub, on="ProbeID", how="inner")
    .drop(columns=["ProbeID"])
    .groupby("Gene")
    .mean()
)
expr_gene.index.name = "Gene"

print(f"\n合并前探针矩阵大小：{expr_probe.shape}")
print(f"\n合并后基因矩阵大小：{expr_gene.shape}")
expr_gene.head()


合并前探针矩阵大小：(62976, 296)

合并后基因矩阵大小：(34949, 296)


,GSM1823702,GSM1823703,GSM1823704,GSM1823705,GSM1823706,GSM1823707,GSM1823708,GSM1823709,GSM1823710,GSM1823711,...,GSM1823988,GSM1823989,GSM1823990,GSM1823991,GSM1823992,GSM1823993,GSM1823994,GSM1823995,GSM1823996,GSM1823997
Gene,,,,,,,,,,,,,,,,,,,,,
3xSLv1,4.859722,12.525190,7.037224,3.741905,5.362853,3.334818,2.497956,2.779081,7.796997,7.336628,...,3.648667,12.009975,9.706994,3.707554,3.374568,5.077706,32.932105,6.151402,3.950380,3.910553
A1BG,535.516700,3634.794000,886.099600,365.293300,896.854300,412.207400,168.935200,131.103600,243.210900,516.196100,...,1536.899000,4000.829000,3179.077000,2973.334000,1440.068000,4051.989000,1990.278000,1139.458000,2208.107000,2587.488000
A1CF,5.940460,38.023283,42.813183,3.697340,6.245011,4.294966,2.726413,3.396612,24.271379,8.732929,...,7.980009,64.482581,37.320740,9.021566,8.277351,16.197230,11.385321,12.521384,11.744898,14.279073
A2BP1,5.685579,44.408785,14.550340,4.955665,4.173715,14.433945,2.490869,4.762691,14.933885,7.389295,...,3.343588,31.948265,17.705430,6.525621,8.170497,16.540025,9.026412,12.367065,5.803522,7.090848
A2LD1,227.191300,1052.650000,578.271400,354.348900,310.102600,201.992800,64.668960,156.852300,255.401400,269.741000,...,163.324900,782.620200,679.497600,407.701600,177.495800,647.335200,407.388900,408.092500,204.237700,206.043100


In [16]:
probe_total = expr_probe.shape[0]
probe_annotated = gpl_sub['ProbeID'].nunique()
probe_matched = expr_probe.reset_index().merge(gpl_sub, on="ProbeID", how="inner")["ProbeID"].nunique()
print(f"总探针数: {probe_total}")
print(f"有注释的探针数: {probe_annotated}")
print(f"表达矩阵和注释能匹配的探针数: {probe_matched}")
print(f"去除未注释后占比: {probe_matched / probe_total:.2%}")

总探针数: 62976
有注释的探针数: 42545
表达矩阵和注释能匹配的探针数: 42545
去除未注释后占比: 67.56%


In [17]:
expr_probe = expr_probe.T
expr_gene = expr_gene.T

In [18]:
print("转置后形状（行=样本，列=基因）:", expr_gene.shape)
print(expr_gene.head())

转置后形状（行=样本，列=基因）: (296, 34949)
Gene           3xSLv1       A1BG       A1CF      A2BP1      A2LD1         A2M  \
GSM1823702   4.859722   535.5167   5.940460   5.685579   227.1913   46960.871   
GSM1823703  12.525190  3634.7940  38.023283  44.408785  1052.6500  156432.520   
GSM1823704   7.037224   886.0996  42.813183  14.550340   578.2714   84857.218   
GSM1823705   3.741905   365.2933   3.697340   4.955665   354.3489   39616.956   
GSM1823706   5.362853   896.8543   6.245011   4.173715   310.1026   40687.011   

Gene            A2ML1     A4GALT      A4GNT  AA081107  ...  tcag7.1213  \
GSM1823702   4.702429  4558.7640   3.904452  1774.270  ...    4.742049   
GSM1823703  78.048350  5997.5070  39.178900  2558.642  ...   14.290600   
GSM1823704  30.520820  7849.2175   8.567911  2632.611  ...   26.760020   
GSM1823705   3.332520  2611.2865   5.401225  1206.235  ...    3.832760   
GSM1823706   3.996803  3422.1065   3.836926  1333.389  ...   13.636010   

Gene        tcag7.1227  tcag7.1231  t

In [19]:
# —— 保存结果 —— 
expr_probe.to_csv(breast_expression_probe)
expr_gene.to_csv(breast_expression_gene)
print(f"已保存两个矩阵文件：{breast_expression_probe}, {breast_expression_gene}")

已保存两个矩阵文件：C:\Users\26494\GA\data\Breast_Cancer/expression_matrix_probe.csv, C:\Users\26494\GA\data\Breast_Cancer/expression_matrix_gene.csv
